In [2]:
import pandas as pd

In [3]:
df=pd.read_csv('silver-price-last-ten-years.csv')
df.head()

,Date,Close
0,16-05-2011,34.35
1,17-05-2011,34.28
2,18-05-2011,34.59
3,19-05-2011,35.75
4,20-05-2011,34.80


In [4]:
df.isna().sum()

Date     0
Close    0
dtype: int64

In [15]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM

# Load and preprocess data
data = pd.read_csv('silver-ten-years.csv')
data['Date'] = pd.to_datetime(data['Date'])
data.set_index('Date', inplace=True)

# Select the relevant column (e.g., 'Close' price)
prices = data[['Close']]

# Normalize the data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(prices)

# Prepare the dataset for LSTM
def create_dataset(dataset, look_back=1):
    X, Y = [], []
    for i in range(len(dataset) - look_back):
        X.append(dataset[i:(i + look_back), 0])
        Y.append(dataset[i + look_back, 0])
    return np.array(X), np.array(Y)

look_back = 60  # Use last 60 days to predict the next day
X, Y = create_dataset(scaled_data, look_back)

# Reshape input to be [samples, time steps, features]
X = np.reshape(X, (X.shape[0], X.shape[1], 1))

# Split into train and test sets
train_size = int(len(X) * 0.8)
X_train, X_test = X[0:train_size], X[train_size:]
Y_train, Y_test = Y[0:train_size], Y[train_size:]

# Build and train the LSTM model
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(look_back, 1)))
model.add(LSTM(50, return_sequences=False))
model.add(Dense(25))
model.add(Dense(1))

model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(X_train, Y_train, batch_size=1, epochs=1)

# Predict the next five days' prices
next_days = 5
last_days = scaled_data[-look_back:]
predictions = []

for _ in range(next_days):
    last_days = np.reshape(last_days, (1, look_back, 1))
    next_day_prediction = model.predict(last_days)
    predictions.append(next_day_prediction[0][0])
    next_day_prediction = np.reshape(next_day_prediction, (1, 1, 1))
    last_days = np.append(last_days[:, 1:, :], next_day_prediction, axis=1)

# Invert the predictions
predictions = scaler.inverse_transform(np.array(predictions).reshape(-1, 1))

# Generate future dates
last_date = data.index[-1]
future_dates = pd.date_range(start=last_date + pd.Timedelta(days=1), periods=next_days)

# Print the next five days' predicted prices with their dates
for date, price in zip(future_dates, predictions.flatten()):
    print(f"{date.date()}: {price:.2f}")


C:\Users\Admin\AppData\Local\Temp\ipykernel_1368\3097165616.py:10: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  data['Date'] = pd.to_datetime(data['Date'])
C:\Users\Admin\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\rnn\rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


2618/2618 ━━━━━━━━━━━━━━━━━━━━ 89s 33ms/step - loss: 0.0027
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 654ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
2024-05-22: 31.62
2024-05-23: 32.04
2024-05-24: 32.29
2024-05-25: 32.45
2024-05-26: 32.56


In [ ]:
# Make predictions
train_predict = model.predict(X_train)
test_predict = model.predict(X_test)

# Invert predictions
train_predict = scaler.inverse_transform(train_predict)
test_predict = scaler.inverse_transform(test_predict)

# Plot the results
train_plot = np.empty_like(scaled_data)
train_plot[:, :] = np.nan
train_plot[look_back:len(train_predict) + look_back, :] = train_predict

test_plot = np.empty_like(scaled_data)
test_plot[:, :] = np.nan
test_plot[len(train_predict) + (look_back):len(train_predict) + (look_back) + len(test_predict), :] = test_predict

plt.figure(figsize=(12, 6))
plt.plot(scaler.inverse_transform(scaled_data), label='True Price')
plt.plot(train_plot, label='Train Predict')
plt.plot(test_plot, label='Test Predict')
plt.legend()
plt.show()

In [ ]:
# Save the model to a file
model.save('silver_price_lstm_model.h5')

In [16]:
from tensorflow.keras.models import load_model

# Load the model from the file
loaded_model = load_model('silver_price_lstm_model.h5')

# Use the loaded model to make predictions
loaded_model_predict = loaded_model.predict(X_test)

21/21 ━━━━━━━━━━━━━━━━━━━━ 2s 50ms/step


In [2]:
data = pd.read_csv('training_data\gold-ten-years.csv')
data.isna().sum()

<>:1: SyntaxWarning: invalid escape sequence '\g'
<>:1: SyntaxWarning: invalid escape sequence '\g'
C:\Users\Admin\AppData\Local\Temp\ipykernel_21500\813429957.py:1: SyntaxWarning: invalid escape sequence '\g'
  data = pd.read_csv('training_data\gold-ten-years.csv')


Date     10
Close    10
dtype: int64

In [4]:
data.dropna(inplace=True)

In [5]:
data.isna().sum()

Date     0
Close    0
dtype: int64

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM

# Load and preprocess data
data = pd.read_csv('training_data/gold-ten-years.csv')
data['Date'] = pd.to_datetime(data['Date'], errors='coerce')
data = data.dropna(subset=['Date'])  # Drop rows where 'Date' could not be parsed
data.set_index('Date', inplace=True)

# Select the relevant column (e.g., 'Close' price)
prices = data[['Close']]

# Normalize the data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(prices)

# Prepare the dataset for LSTM
def create_dataset(dataset, look_back=1):
    X, Y = [], []
    for i in range(len(dataset) - look_back):
        X.append(dataset[i:(i + look_back), 0])
        Y.append(dataset[i + look_back, 0])
    return np.array(X), np.array(Y)

look_back = 60  # Use last 60 days to predict the next day
X, Y = create_dataset(scaled_data, look_back)

# Reshape input to be [samples, time steps, features]
X = np.reshape(X, (X.shape[0], X.shape[1], 1))

# Split into train and test sets
train_size = int(len(X) * 0.8)
X_train, X_test = X[0:train_size], X[train_size:]
Y_train, Y_test = Y[0:train_size], Y[train_size:]

# Build and train the LSTM model
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(look_back, 1)))
model.add(LSTM(50, return_sequences=False))
model.add(Dense(25))
model.add(Dense(1))

model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(X_train, Y_train, batch_size=1, epochs=1)

# Predict the next five days' prices
next_days = 5
last_days = scaled_data[-look_back:]
predictions = []

for _ in range(next_days):
    last_days = np.reshape(last_days, (1, look_back, 1))
    next_day_prediction = model.predict(last_days)
    predictions.append(next_day_prediction[0][0])
    next_day_prediction = np.reshape(next_day_prediction, (1, 1, 1))
    last_days = np.append(last_days[:, 1:, :], next_day_prediction, axis=1)

# Invert the predictions
predictions = scaler.inverse_transform(np.array(predictions).reshape(-1, 1))

# Generate future dates
last_date = data.index[-1]
if pd.isna(last_date):
    raise ValueError("The last date in the dataset is not valid.")
future_dates = pd.date_range(start=last_date + pd.Timedelta(days=1), periods=next_days)

# Print the next five days' predicted prices with their dates
for date, price in zip(future_dates, predictions.flatten()):
    print(f"{date.date()}: {price:.2f}")


C:\Users\Admin\AppData\Local\Temp\ipykernel_18072\812972199.py:10: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  data['Date'] = pd.to_datetime(data['Date'], errors='coerce')
C:\Users\Admin\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\rnn\rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


2009/2009 ━━━━━━━━━━━━━━━━━━━━ 113s 52ms/step - loss: 0.0032
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 910ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step
2024-05-22: 2287.17
2024-05-23: 2273.66
2024-05-24: 2251.97
2024-05-25: 2227.11
2024-05-26: 2201.48


In [2]:
# Save the model to a file
model.save('gold_price_lstm_model.h5')